In [3]:
# Définition des matrices 3x3
matrice1 = [1 2 3;
            4 5 6;
            7 8 9]

matrice2 = [9 8 7;
            6 5 4;
            3 2 1]

# Multiplication des matrices
produit = matrice1 * matrice2

# Calcul de la somme des coefficients du produit
somme = sum(produit)

# Affichage du résultat
println("Le produit des matrices est :")
println(produit)
println("La somme des coefficients du produit est : ", somme)


Le produit des matrices est :
[30 24 18; 84 69 54; 138 114 90]
La somme des coefficients du produit est : 621
